In [1]:
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np
import datetime
from datetime import date
import time
from urllib.request import Request, urlopen

import urllib.request
from bs4 import BeautifulSoup
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

credentials = {'CREDENTIALS_HERE'}
gc = gspread.service_account_from_dict(credentials)

In [2]:
url = 'https://aveda.edu/locations/'
req = urllib.request.Request(url)
response = urllib.request.urlopen(req).read()
soup = BeautifulSoup(response)

In [7]:
all_locations = soup.find_all('div', class_='location-tile')

In [46]:
aveda_dict = {}
aveda_df = pd.DataFrame()

for i in range(len(all_locations)):
    aveda_dict['Website'] = all_locations[i].find(href=True)['href']
    state = all_locations[i].find('div', class_='label').text
    aveda_dict['State'] = state
    city = all_locations[i].find('span', class_='city').text
    aveda_dict['City'] = city
    # aveda_dict['Phone'] = all_locations[i].find('span', class_='phone').text
    aveda_dict['Address'] = all_locations[i].find('span', class_='address').text + ' - ' + city + ', ' + state
    school_name = all_locations[i].find('span', class_='school').text
    if city in school_name:
        aveda_dict['School Name'] = school_name.replace(city, ' - '+ city).replace('  ',' ').replace(' - - ', ' - ').replace('(', '').replace(')', '')
    else:
        aveda_dict['School Name'] = school_name + ' - ' + city

    aveda_df = aveda_df.append(aveda_dict, ignore_index=True)

aveda_df = aveda_df.reindex(['Company ID', 'School Name', 'City', 'State', 'Address', 'Website'], axis=1)

In [47]:
aveda_df.head(50)

,Company ID,School Name,City,State,Address,Website
0,NaN,Aveda Arts - Birmingham,Birmingham,Alabama,"3200 Galleria Blvd - Birmingham, Alabama",https://aveda.edu/locations/location/birmingha...
1,NaN,Aveda Institute Phoenix - Tempe,Tempe,Arizona,"8475 South Emerald Drive - Suite 101, 104 - Te...",https://aveda.edu/locations/location/phoenix-az
2,NaN,Aveda Institute - Tucson,Tucson,Arizona,"145 S. 6th Ave - Tucson, Arizona",https://aveda.edu/locations/location/tucson-az
3,NaN,Aveda Institute - Los Angeles,Los Angeles,California,"10935 Weyburn Avenue - Los Angeles, California",https://aveda.edu/locations/location/los-angel...
4,NaN,Cinta Aveda Institute - San Francisco,San Francisco,California,"305 Kearny St - San Francisco, California",https://aveda.edu/locations/location/san-franc...
5,NaN,Cinta Aveda Institute - San Jose,San Jose,California,"111 West St. John Street - San Jose, California",https://aveda.edu/locations/location/san-jose-ca
6,NaN,Aveda Institute - Denver,Denver,Colorado,"700 16th Street - Denver, Colorado",https://aveda.edu/locations/location/denver-co
7,NaN,Aveda Institute - Brandon,Brandon,Florida,"1516 West Brandon Boulevard - Brandon, Florida",https://aveda.edu/locations/location/brandon-fl
8,NaN,Aveda Institute Tampa Bay - Clearwater,Clearwater,Florida,"28272 US Highway 19 N - Clearwater, Florida",https://aveda.edu/locations/location/tampa-bay-fl
9,NaN,Aveda Institute South Florida - Davie,Davie,Florida,"4186 South University Drive - Davie, Florida",https://aveda.edu/locations/location/davie-fl


In [49]:
aveda_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1inma86cQKoPFBZX2txtj6oXqL3VYN58eT0frUTdLRQs/edit#gid=0')
aveda_sheet = aveda_wb.worksheet('Sheet1')

In [50]:
set_with_dataframe(aveda_sheet, aveda_df)